# Dataset download


In [ ]:
from google.colab import files 
data_to_load = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-49585a60-3b11-c3b3-5a41-8bac97c1dcac)


In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json


kaggle.json


In [ ]:
!kaggle datasets download -d nih-chest-xrays/data

!unzip \*.zip  && rm *.zip

 33% 13.8G/42.0G [02:01<04:09, 122MB/s]
User cancelled operation
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/_monitor.py", line 44, in exit
    self.join()
  File "/usr/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
Archive:  data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of *.zip or
        *.zip.zip, and cannot find data.zip.ZIP, period.

No zipfiles found.


# Dependecies

In [ ]:
import os
import json
import logging
import argparse
from glob import glob
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
import time
import tqdm
from sklearn import metrics
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset
from PIL import Image
from torch.optim import SGD, Adadelta, Adagrad, Adam, RMSprop
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
%matplotlib inline
# from global_pool import GlobalPool
# from attention_map import AttentionMap,CAModule
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Loading data

In [ ]:
all_xray_df = pd.read_csv('Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('images*', '*', '*.png'))}
# all_image_paths
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['Patient Age'] = all_xray_df['Patient Age'].map(lambda x: int(x))
# all_xray_df.sample(3)

Scans found: 112120 , Total Headers 112120


In [ ]:
112120*.1

11212.0

In [ ]:
# 11212/len(train_val_df)

In [ ]:
# all_xray_df = pd.read_csv('Data_Entry_2017.csv')

In [ ]:
# all_xray_df.head()

In [ ]:
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
# all_xray_df

All Labels (14): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [ ]:
all_labels



['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Effusion',
 'Emphysema',
 'Fibrosis',
 'Hernia',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pleural_Thickening',
 'Pneumonia',
 'Pneumothorax']

In [ ]:
len(all_xray_df['Image Index'].unique())

112120

In [ ]:
len(all_xray_df['Image Index'])

112120

In [ ]:
#### Disease distribution visualization
# sample_weights = all_xray_df['Finding Labels'].map(lambda x: len(x.split('|')) if len(x)>0 else 0).values + 4e-2
# sample_weights /= sample_weights.sum()
# all_xray_df = all_xray_df.sample(40000, weights=sample_weights)

# label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
# fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
# ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
# ax1.set_xticks(np.arange(len(label_counts))+0.5)
# _ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
train_list_df = pd.read_csv("train_val_list.txt", sep=" ", header=None, 
                 names=["names"])
test_list_df = pd.read_csv("test_list.txt", sep=" ", header=None, 
                 names=["names"])
train_val_df = all_xray_df[all_xray_df['Image Index'].isin(train_list_df.names.values)] 
test_df = all_xray_df[all_xray_df['Image Index'].isin(test_list_df.names.values)] 
train_val_df = train_val_df.reset_index().drop('index',axis = 1)
test_df = test_df.reset_index().drop('index',axis = 1)

In [ ]:
25596/(112120)


0.22829111666072066

In [ ]:
len(train_list_df) +25596

112120

#

In [ ]:
## Split na kore try korte paro
train_df,val_df = train_test_split(train_val_df, test_size=0.111, random_state=42)

In [ ]:
train_df =train_df.reset_index().drop('index',axis = 1)
val_df = val_df.reset_index().drop('index',axis = 1)

# Creating DataLoader

In [ ]:



class ChestXrayDataSet(Dataset):
    def __init__(self,data_frame, transform=None):
        """
        Args:
            data_dir: path to image directory.
            image_list_file: path to the file containing images
                with corresponding labels.
            transform: optional transform to be applied on a sample.
        """
        self.data_frame = data_frame
        self.transform = transform

    def __getitem__(self, index):
        """
        Args:
            index: the index of item
        Returns:
            image and its labels
        """
        image_name = self.data_frame['path'][index]
        image = Image.open(image_name).convert('RGB')
        label = self.data_frame[all_labels].iloc[[index]].values
        if self.transform is not None:
            image = self.transform(image)
        return image, torch.FloatTensor(label).t() ,image_name

    def __len__(self):
        return len(self.data_frame)


In [ ]:
train_transform=transforms.Compose([transforms.Resize((256, 256)),
                              transforms.RandomHorizontalFlip(p=0.5),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])
test_transform = transforms.Compose([transforms.Resize((256, 256)),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])
                          
train_dataset  = ChestXrayDataSet(train_df,train_transform)
val_dataset = ChestXrayDataSet(val_df,test_transform)
test_dataset = ChestXrayDataSet(test_df,test_transform)

In [ ]:
.

0.08000000000000002

In [ ]:
##
BATCH_SIZE = 16 ##Used in paper(Higher batch size resulted lower auc)

dataloader_train = torch.utils.data.DataLoader(
       train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0)
dataloader_dev = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=0)


In [ ]:
# a,b,c = train_dataset[0]
N_CLASSES = len(all_labels)
N_CLASSES

14

In [ ]:
def compute_AUCs(gt, pred):
    """Computes Area Under the Curve (AUC) from prediction scores.
    Args:
        gt: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          true binary labels.
        pred: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          can either be probability estimates of the positive class,
          confidence values, or binary decisions.
    Returns:
        List of AUROCs of all classes.
    """
    AUROCs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(N_CLASSES):
        AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    return AUROCs

# Training and testing one epoch

In [ ]:
from tqdm.notebook import tqdm
criterion = torch.nn.BCEWithLogitsLoss(reduction = 'none')
def train_epoch(summary, summary_dev, cfg, model, dataloader,
                dataloader_dev, optimizer,  best_dict,
                dev_header):
    torch.set_grad_enabled(True)
    model.train()
    
    device = torch.device('cuda')
    steps = len(dataloader)
    dataiter = iter(dataloader)
    num_tasks = 14
    label_header = all_labels
    #num_tasks = len(cfg.num_classes)
    scaler = torch.cuda.amp.GradScaler()
    time_now = time.time()
    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)
    for step in tqdm(range(steps)):
        image, target,_ = next(dataiter)
        image = image.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast() :
          output = model(image)
          loss_t = criterion(output.float(), target.float()).sum(dim = 0) 
          loss_sum += np.array([i.item() for i in loss_t])
          loss =  criterion(output.float(), target.float()).mean()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        output = f.sigmoid(output)
        label = output.ge(0.5).float()
        acc_t = (target == label).float().sum(dim=0) / (label.shape[0])
        acc_sum += np.array([i.item() for i in acc_t])
        summary['step'] += 1
        
        if summary['step'] % cfg.log_every == 0:
            time_spent = time.time() - time_now
            time_now = time.time()

            loss_sum /= cfg.log_every
            acc_sum /= cfg.log_every
            loss_str = ' '.join(map(lambda x: '{:.5f}'.format(x), loss_sum))
            acc_str = ' '.join(map(lambda x: '{:.3f}'.format(x), acc_sum))

            print(
                '{}, Train, Epoch : {}, Step : {}, Loss : {}, '
                'Acc : {}, Run Time : {:.2f} sec'
                .format(time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['epoch'] + 1, summary['step'], loss_str,
                        acc_str, time_spent))
            
            loss_sum = np.zeros(num_tasks)
            acc_sum = np.zeros(num_tasks)

        if summary['step'] % cfg.test_every == 999999999:
            time_now = time.time()
            summary_dev, predlist, true_list = test_epoch(
                summary_dev, cfg, model, dataloader_dev)
            time_spent = time.time() - time_now

            auclist = []
            for i in range(len(cfg.num_classes)):
                y_pred = predlist[i]
                y_true = true_list[i]
                fpr, tpr, thresholds = metrics.roc_curve(
                    y_true, y_pred, pos_label=1)
                auc = metrics.auc(fpr, tpr)
                auclist.append(auc)
            summary_dev['auc'] = np.array(auclist)

            loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                        summary_dev['loss']))
            acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['acc']))
            auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                                       summary_dev['auc']))

            print(
                '{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {},'
                'Mean auc: {:.3f} ''Run Time : {:.2f} sec' .format(
                    time.strftime("%Y-%m-%d %H:%M:%S"),
                    summary['step'],
                    loss_dev_str,
                    acc_dev_str,
                    auc_dev_str,
                    summary_dev['auc'].mean(),
                    time_spent))

            save_best = False
            mean_acc = summary_dev['acc'][cfg.save_index].mean()
            if mean_acc >= best_dict['acc_dev_best']:
                best_dict['acc_dev_best'] = mean_acc
                if cfg.best_target == 'acc':
                    save_best = True

            mean_auc = summary_dev['auc'][cfg.save_index].mean()
            if mean_auc >= best_dict['auc_dev_best']:
                best_dict['auc_dev_best'] = mean_auc
                if cfg.best_target == 'auc':
                    save_best = True

            mean_loss = summary_dev['loss'][cfg.save_index].mean()
            if mean_loss <= best_dict['loss_dev_best']:
                best_dict['loss_dev_best'] = mean_loss
                if cfg.best_target == 'loss':
                    save_best = True

            if save_best:
                torch.save(
                    {'epoch': summary['epoch'],
                     'step': summary['step'],
                     'acc_dev_best': best_dict['acc_dev_best'],
                     'auc_dev_best': best_dict['auc_dev_best'],
                     'loss_dev_best': best_dict['loss_dev_best'],
                     'state_dict': model.state_dict()},
                    os.path.join(cfg.save_path, 'best{}.tar'.format(
                        best_dict['best_idx']))
                )
                best_dict['best_idx'] += 1
                if best_dict['best_idx'] > cfg.save_top_k:
                    best_dict['best_idx'] = 1
                print(
                    '{}, Best, Step : {}, Loss : {}, Acc : {},Auc :{},'
                    'Best Auc : {:.3f}' .format(
                        time.strftime("%Y-%m-%d %H:%M:%S"),
                        summary['step'],
                        loss_dev_str,
                        acc_dev_str,
                        auc_dev_str,
                        best_dict['auc_dev_best']))
        model.train()
        torch.set_grad_enabled(True)
    summary['epoch'] += 1

    return summary, best_dict

In [ ]:
def test_epoch(summary, cfg, model, dataloader):
    torch.set_grad_enabled(False)
    model.eval()

    device = torch.device('cuda')
    steps = len(dataloader)
    dataiter = iter(dataloader)
    num_tasks = len(cfg.num_classes)

    loss_sum = np.zeros(num_tasks)
    acc_sum = np.zeros(num_tasks)
    predlist = list()
    true_list = list()
    gt = torch.FloatTensor()
    gt = gt.cuda()
    pred = torch.FloatTensor()
    pred = pred.cuda()
    scaler = torch.cuda.amp.GradScaler()
    for step in tqdm(range(steps)):
        image, target,_ = next(dataiter)
        image = image.to(device)
        target = target.to(device)
        output = model(image)
        output = f.sigmoid(output)
        gt = torch.cat((gt, target), 0)
        pred = torch.cat((pred, output), 0)

        if step == 0: 
            output_t = output
            target_t = target
        else :
            output_t = torch.cat([output_t, output],dim = 0)
            target_t = torch.cat([target_t, target],dim = 0)
        # different number of tasks

        loss_t = criterion(output.float(), target.float()).sum(dim = 0) 

        loss_sum += np.array([i.item() for i in loss_t])
       
        label = output.ge(0.5).float()
        acc_t = (target == label).float().sum(dim=0) / (label.shape[0])
        acc_sum += np.array([i.item() for i in acc_t])
       
    summary['loss'] = loss_sum / steps
    summary['acc'] = acc_sum / steps
    predlist = [output_t[:,i].cpu().numpy() for i in range(output_t.shape[1])]
    true_list = [target_t[:,i].cpu().numpy() for i in range(target_t.shape[1])]
    return summary, predlist, true_list,gt,pred

#Normalization and optimizer

In [ ]:


# def get_optimizer(params, cfg):
#     if cfg.optimizer == 'SGD':
#         return SGD(params, lr=cfg.lr, momentum=cfg.momentum,
#                    weight_decay=cfg.weight_decay)
#     elif cfg.optimizer == 'Adadelta':
#         return Adadelta(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
#     elif cfg.optimizer == 'Adagrad':
#         return Adagrad(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
#     elif cfg.optimizer == 'Adam':
#         return Adam(params, lr=cfg.lr, weight_decay=cfg.weight_decay)
#     elif cfg.optimizer == 'RMSprop':
#         return RMSprop(params, lr=cfg.lr, momentum=cfg.momentum,
#                        weight_decay=cfg.weight_decay)
#     else:
#         raise Exception('Unknown optimizer : {}'.format(cfg.optimizer))

# Configuration

In [ ]:
from easydict import EasyDict as edict
cfg = {
    "num_classes":[1,1,1,1,1,1,1,1,1,1,1,1,1,1],
    "batch_weight": True,
    "pos_weight": [1,1,1,1,1,1,1,1,1,1,1,1,1,1], ## class weight diye try kora jete pare
    "pretrained": True,
    "log_every": 100,
    "test_every": 1803,
    "epoch": 50,
    "norm_type": "BatchNorm",
    "global_pool": "AVG_MAX",
    "fc_bn": False,
    "lse_gamma": 0.5,
    "fc_drop": 0,
    "optimizer": "Adam",
    "criterion": "BCE",
    "lr": 0.01,
    "lr_factor": 0.1,
    "lr_epochs": [2],
    "momentum": 0.9,
    "weight_decay": 0.0,
    "best_target": "auc",
    "save_top_k": 30,
    'save_path' : '/content/drive/MyDrive/Colab Notebooks/Chexpert/Densenet121_Baseline_adam',
    'logtofile' : True,
    "best_target": "auc",
    "save_top_k": 30,
    "save_index": [0,1,2,3,4,5,6,7,8,9,10,11,12,13],
    'save_path' : '/content/drive/MyDrive/Colab Notebooks/Chexpert/Densenet121_Baseline_adam',
    'logtofile' : True,
    'resume' : False,
    'pre_train' : None,
    'verbose' :  False
}
cfg  = edict(cfg)

# Model

In [ ]:


# class Classifier_Pool(nn.Module) :
#     def __init__(self, n_class) :
#         super(Classifier_Pool,self).__init__()
#         self.n_class = n_class
#         for i in range(self.n_class):
#             setattr(self,'conv_'+str(i+1), nn.Conv2d(1024*2,1, kernel_size=1,stride = 1, padding = 0))
            
#         self.input_layer = 128*self.n_class
#         self.linear = nn.Linear(self.input_layer,n_class)
#         self.sigmoid = nn.Sigmoid()
#         self.global_pool = GlobalPool(cfg)
#         for i in range(self.n_class):
#             setattr(self,'attention_'+str(i+1), CAModule(1024))
#     def forward(self,feat_map):
#         logits = list()
#         logit_maps = list()
#         for i in range(self.n_class) :
        
#             classifier = getattr(self, 'conv_' + str(i+1))
#             attention = getattr(self, 'attention_'+str(i+1))
#             feat_map = attention(feat_map)
#             logit_map = None
#             logit_maps.append(logit_map)
#             feat = self.global_pool(feat_map, logit_map)
#             logit = classifier(feat)
#             logit = logit.squeeze(-1)
#             logits.append(logit)
#         output = torch.cat(logits,dim = 1)                      
#         return output

In [ ]:

class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        self.densenet121.classifier = nn.Sequential(
                        nn.Linear(1024, 14)          
                        )
    def forward(self, x):
        x = self.densenet121(x)
        

        return x.unsqueeze(dim=2)


In [ ]:
model = DenseNet121()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [ ]:
model(torch.rand(5,3,256,256)).shape

torch.Size([5, 14, 1])

# Training loop

In [ ]:


if not os.path.exists(cfg.save_path):
    os.mkdir(cfg.save_path)
if cfg.logtofile is True:
    logging.basicConfig(filename=cfg.save_path + '/log.txt',
                        filemode="w", level=logging.INFO)
else:
    logging.basicConfig(level=logging.INFO)





device = torch.device('cuda')
model = DenseNet121()
learning_rate = 0.001#used in paper
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
Lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')## USed in paper
#Lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size = 15, gamma = .1)
model = model.to(device).train()
if cfg.pre_train is not None:
    if os.path.exists(cfg.pre_train):
        ckpt = torch.load(cfg.pre_train, map_location=device)
        model.module.load_state_dict(ckpt)




# dataloader_train = DataLoader(
#     CheXDataset(cfg.train_csv, diseases, data_aug_transforms),
#     batch_size=cfg.train_batch_size, num_workers=0,
#     drop_last=True, shuffle=True)
# dataloader_dev = DataLoader(
#     CheXDataset(cfg.dev_csv, diseases),
#     batch_size=cfg.dev_batch_size, num_workers=0,
#     drop_last=False, shuffle=False)
dev_header = all_labels

summary_train = {'epoch': 0, 'step': 0}
summary_dev = {'loss': float('inf'), 'acc': 0.0}
epoch_start = 0
best_dict = {
    "acc_dev_best": 0.0,
    "auc_dev_best": 0.0,
    "loss_dev_best": float('inf'),
    "fused_dev_best": 0.0,
    "best_idx": 1}

if cfg.resume:
    ckpt_path = os.path.join(cfg.save_path, 'train.tar')
    ckpt = torch.load(ckpt_path, map_location=device)
    model.module.load_state_dict(ckpt['state_dict'])
    summary_train = {'epoch': ckpt['epoch'], 'step': ckpt['step']}
    best_dict['acc_dev_best'] = ckpt['acc_dev_best']
    best_dict['loss_dev_best'] = ckpt['loss_dev_best']
    best_dict['auc_dev_best'] = ckpt['auc_dev_best']
    epoch_start = ckpt['epoch']

for epoch in range(epoch_start, cfg.epoch):
    # lr = lr_schedule(cfg.lr, cfg.lr_factor, summary_train['epoch'],
    #                  cfg.lr_epochs)
    

    summary_train, best_dict = train_epoch(
        summary_train, summary_dev, cfg, model,
        dataloader_train, dataloader_dev, optimizer,
         best_dict, dev_header)
    # Lr_scheduler.step()
    time_now = time.time()
    summary_dev, predlist, true_list,gt,pred = test_epoch(
        summary_dev, cfg, model, dataloader_dev)
    time_spent = time.time() - time_now


    

    auclist = []
    for i in range(len(cfg.num_classes)):
        y_pred = predlist[i]
        y_true = true_list[i]
        fpr, tpr, thresholds = metrics.roc_curve(
            y_true, y_pred, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        auclist.append(auc)
    summary_dev['auc'] = np.array(auclist)

    loss_dev_str = ' '.join(map(lambda x: '{:.5f}'.format(x),
                                summary_dev['loss']))
    acc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['acc']))
    auc_dev_str = ' '.join(map(lambda x: '{:.3f}'.format(x),
                               summary_dev['auc']))

    print(
        '{}, Dev, Step : {}, Loss : {}, Acc : {}, Auc : {},'
        'Mean auc: {:.3f} ''Run Time : {:.2f} sec' .format(
            time.strftime("%Y-%m-%d %H:%M:%S"),
            summary_train['step'],
            loss_dev_str,
            acc_dev_str,
            auc_dev_str,
            summary_dev['auc'].mean(),
            time_spent))

    
    AUROCs = compute_AUCs(gt, pred)
    AUROC_avg = np.array(AUROCs).mean()
    print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
    for i in range(14):
        print('The AUROC of {} is {}'.format(all_labels[i], AUROCs[i]))
    save_best = False

    mean_acc = summary_dev['acc'][cfg.save_index].mean()
    if mean_acc >= best_dict['acc_dev_best']:
        best_dict['acc_dev_best'] = mean_acc
        if cfg.best_target == 'acc':
            save_best = True

    mean_auc = summary_dev['auc'][cfg.save_index].mean()
    if mean_auc >= best_dict['auc_dev_best']:
        best_dict['auc_dev_best'] = mean_auc
        if cfg.best_target == 'auc':
            save_best = True

    mean_loss = summary_dev['loss'][cfg.save_index].mean()
    if mean_loss <= best_dict['loss_dev_best']:
        best_dict['loss_dev_best'] = mean_loss
        if cfg.best_target == 'loss':
            save_best = True

    if save_best:
        torch.save(
            {'epoch': summary_train['epoch'],
             'step': summary_train['step'],
             'acc_dev_best': best_dict['acc_dev_best'],
             'auc_dev_best': best_dict['auc_dev_best'],
             'loss_dev_best': best_dict['loss_dev_best'],
             'state_dict': model.state_dict()},
            os.path.join(cfg.save_path,
                         'best{}.tar'.format(best_dict['best_idx']))
        )
        best_dict['best_idx'] += 1
        if best_dict['best_idx'] > cfg.save_top_k:
            best_dict['best_idx'] = 1
        print(
            '{}, Best, Step : {}, Loss : {}, Acc : {},'
            'Auc :{},Best Auc : {:.3f}' .format(
                time.strftime("%Y-%m-%d %H:%M:%S"),
                summary_train['step'],
                loss_dev_str,
                acc_dev_str,
                auc_dev_str,
                best_dict['auc_dev_best']))
    torch.save({'epoch': summary_train['epoch'],
                'step': summary_train['step'],
                'acc_dev_best': best_dict['acc_dev_best'],
                'auc_dev_best': best_dict['auc_dev_best'],
                'loss_dev_best': best_dict['loss_dev_best'],
                'state_dict': model.state_dict()},
               os.path.join(cfg.save_path, 'train.tar'))


  0%|          | 0/4808 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


2022-08-30 19:29:59, Train, Epoch : 1, Step : 100, Loss : 5.67151 2.15833 1.95514 1.40387 6.03846 1.85033 1.63117 0.44666 7.65322 3.43760 3.68771 2.22743 1.32379 2.57041, Acc : 0.889 0.968 0.975 0.985 0.866 0.973 0.978 0.998 0.820 0.940 0.941 0.975 0.981 0.959, Run Time : 68.37 sec
2022-08-30 19:30:56, Train, Epoch : 1, Step : 200, Loss : 4.98311 1.91202 2.33274 1.52465 5.10562 1.31408 1.01098 0.23866 7.40827 3.03393 3.47004 1.55063 1.13284 2.14975, Acc : 0.907 0.976 0.965 0.981 0.900 0.984 0.989 0.998 0.824 0.955 0.946 0.981 0.987 0.971, Run Time : 57.46 sec
2022-08-30 19:31:53, Train, Epoch : 1, Step : 300, Loss : 4.92270 1.76179 2.57536 1.32209 4.66910 1.69601 1.30195 0.22846 6.66598 3.21167 3.45876 1.76513 1.06467 2.41817, Acc : 0.909 0.978 0.963 0.983 0.912 0.979 0.985 0.998 0.854 0.951 0.947 0.978 0.988 0.966, Run Time : 57.21 sec
2022-08-30 19:32:51, Train, Epoch : 1, Step : 400, Loss : 5.07272 1.79102 2.40826 1.16492 5.33344 1.09980 1.00555 0.16126 7.37023 2.85366 3.61474 2.009

  0%|          | 0/400 [00:00<?, ?it/s]

2022-08-30 20:25:21, Dev, Step : 4808, Loss : 46.97081 45.22035 45.42595 45.30284 47.07344 44.99651 44.87991 44.38778 48.07105 46.18047 45.42841 46.86307 44.84864 45.20794, Acc : 0.872 0.958 0.929 0.964 0.818 0.957 0.980 0.997 0.761 0.932 0.937 0.935 0.978 0.896, Auc : 0.558 0.614 0.641 0.705 0.647 0.547 0.676 0.493 0.621 0.550 0.534 0.579 0.571 0.571,Mean auc: 0.593 Run Time : 658.83 sec
The average AUROC is 0.593
The AUROC of Atelectasis is 0.5584196826882841
The AUROC of Cardiomegaly is 0.6143112248760583
The AUROC of Consolidation is 0.6414512801211885
The AUROC of Edema is 0.7047234799145949
The AUROC of Effusion is 0.6472106754813667
The AUROC of Emphysema is 0.5469931254380077
The AUROC of Fibrosis is 0.6759809813308043
The AUROC of Hernia is 0.4925127401019208
The AUROC of Infiltration is 0.6207608189186778
The AUROC of Mass is 0.5496834763763178
The AUROC of Nodule is 0.5344224925047251
The AUROC of Pleural_Thickening is 0.5792295888994328
The AUROC of Pneumonia is 0.571337313

...

  0%|          | 0/400 [00:00<?, ?it/s]

2022-08-31 18:14:37, Dev, Step : 115392, Loss : 46.57139 44.74930 45.61296 45.37909 46.18971 45.25907 44.79925 44.39154 47.46260 45.89116 46.14733 45.64301 44.81514 45.33874, Acc : 0.872 0.960 0.929 0.963 0.831 0.955 0.983 0.997 0.760 0.928 0.936 0.955 0.978 0.896, Auc : 0.740 0.874 0.723 0.833 0.810 0.827 0.771 0.858 0.683 0.785 0.699 0.738 0.675 0.816,Mean auc: 0.774 Run Time : 659.49 sec
The average AUROC is 0.774
The AUROC of Atelectasis is 0.7399322356754114
The AUROC of Cardiomegaly is 0.8744080090732944
The AUROC of Consolidation is 0.7226330532407577
The AUROC of Edema is 0.8332369660406628
The AUROC of Effusion is 0.8098055275833071
The AUROC of Emphysema is 0.827103737208794
The AUROC of Fibrosis is 0.7710392885906714
The AUROC of Hernia is 0.8583751925829359
The AUROC of Infiltration is 0.6827720007677128
The AUROC of Mass is 0.7848941465283177
The AUROC of Nodule is 0.6994035110201379
The AUROC of Pleural_Thickening is 0.7379347078200511
The AUROC of Pneumonia is 0.67517023

  0%|          | 0/4808 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


2022-08-31 18:14:42, Train, Epoch : 25, Step : 115400, Loss : 0.26319 0.07767 0.09609 0.06194 0.27456 0.08487 0.10801 0.00227 0.56559 0.24411 0.26791 0.06734 0.01240 0.16663, Acc : 0.073 0.079 0.079 0.079 0.072 0.078 0.078 0.080 0.066 0.074 0.076 0.079 0.080 0.076, Run Time : 4.83 sec
2022-08-31 18:15:41, Train, Epoch : 25, Step : 115500, Loss : 4.07928 0.89466 1.76316 0.92307 3.21792 0.93345 0.71023 0.14366 6.22084 2.05882 2.71061 1.24667 0.73171 1.52499, Acc : 0.906 0.985 0.969 0.983 0.913 0.983 0.991 0.998 0.842 0.959 0.951 0.979 0.991 0.967, Run Time : 58.53 sec
2022-08-31 18:16:40, Train, Epoch : 25, Step : 115600, Loss : 3.82171 0.63817 1.97673 0.80957 3.09637 0.68971 1.09645 0.02112 5.94012 2.14057 3.33968 1.62611 0.62730 1.59536, Acc : 0.913 0.987 0.964 0.984 0.919 0.989 0.984 1.000 0.854 0.954 0.940 0.973 0.992 0.971, Run Time : 58.67 sec
2022-08-31 18:17:38, Train, Epoch : 25, Step : 115700, Loss : 4.41846 1.01804 1.89604 0.88770 3.56081 0.75001 0.94302 0.20171 6.52196 2.3172